# Demo for PyTorch Mask RCNN
Base Repo:  https://github.com/multimodallearning/pytorch-mask-rcnn

Sample Repo:  https://github.com/michhar/pytorch-mask-rcnn-samples

**Note:  Run this notebook inside of the Sample Repo after cloning**

## Prerequisites

1.  NVIDIA CUDA Toolkit setup ([NVIDIA CUDA Toolkit Docs](https://docs.nvidia.com/cuda/index.html))
2.  PyTorch (tested with version below)

## Installs and Setup

In [38]:
import torch
torch.__version__

'0.4.1'

**Build two extensions**

* nms
* roi align

In [70]:
! ls nms/src/cuda

nms_kernel.cu     nms_kernel.cu.o   nms_kernel.h      nms_kernel.h.bkup


In [77]:
!  export CC="/usr/local/opt/llvm/bin/clang" &&\
cd nms/src/cuda/ && nvcc -c -o nms_kernel.cu.o nms_kernel.cu -x cu -Xcompiler -fPIC -l/usr/local/cuda/lib/libcudart.so &&\
cd ../../ &&\
python3 build.py

/Users/micheleenharris/Documents/bin/github/pytorch-mask-rcnn-samples/nms
generating /var/folders/_f/464k2bb144v0gwg6lys9db780000gn/T/tmpi2s8zzoc/_nms.c
setting the current directory to '/var/folders/_f/464k2bb144v0gwg6lys9db780000gn/T/tmpi2s8zzoc'
running build_ext
building '_nms' extension
creating Users
creating Users/micheleenharris
creating Users/micheleenharris/Documents
creating Users/micheleenharris/Documents/bin
creating Users/micheleenharris/Documents/bin/github
creating Users/micheleenharris/Documents/bin/github/pytorch-mask-rcnn-samples
creating Users/micheleenharris/Documents/bin/github/pytorch-mask-rcnn-samples/nms
creating Users/micheleenharris/Documents/bin/github/pytorch-mask-rcnn-samples/nms/src
/usr/local/opt/llvm/bin/clang -Wno-unused-result -Wsign-compare -Wunreachable-code -fno-common -dynamic -DNDEBUG -g -fwrapv -O3 -Wall -arch x86_64 -g -I/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/torch/utils/ffi/../../lib/include -I/Library/Fr

For the next cuda compiler command, `-arch` flag refers to architectures listed at https://docs.nvidia.com/cuda/cuda-compiler-driver-nvcc/index.html#gpu-compilation.

In [9]:
!  export CC="/usr/local/bin/gcc-7" &&\
cd roialign/roi_align/src/cuda/ &&\
nvcc -c -o crop_and_resize_kernel.cu.o crop_and_resize_kernel.cu -x cu -Xcompiler -fPIC &&\
cd ../../ &&\
python3 build.py

/Users/micheleenharris/Documents/bin/github/pytorch-mask-rcnn-samples/roialign/roi_align
generating /var/folders/_f/464k2bb144v0gwg6lys9db780000gn/T/tmp6hrwq3dc/_crop_and_resize.c
setting the current directory to '/var/folders/_f/464k2bb144v0gwg6lys9db780000gn/T/tmp6hrwq3dc'
running build_ext
building '_crop_and_resize' extension
creating Users
creating Users/micheleenharris
creating Users/micheleenharris/Documents
creating Users/micheleenharris/Documents/bin
creating Users/micheleenharris/Documents/bin/github
creating Users/micheleenharris/Documents/bin/github/pytorch-mask-rcnn-samples
creating Users/micheleenharris/Documents/bin/github/pytorch-mask-rcnn-samples/roialign
creating Users/micheleenharris/Documents/bin/github/pytorch-mask-rcnn-samples/roialign/roi_align
creating Users/micheleenharris/Documents/bin/github/pytorch-mask-rcnn-samples/roialign/roi_align/src
/usr/local/bin//gcc-7 -Wno-unused-result -Wsign-compare -Wunreachable-code -fno-common -dynamic -DNDEBUG -g -fwrapv -O3 -

In [10]:
! ls

FixingPyTorchInstallManual.ipynb fish_pytorch_style.py
LICENSE                          images
README.md                        model.py
Setup_and_Demo.ipynb             nms
Train_Fish.ipynb                 requirements.txt
coco.py                          roialign
config.py                        utils.py
data                             visualize.py
demo.py


In [11]:
! cd roialign/roi_align/_ext/crop_and_resize/ && ls

__init__.py         _crop_and_resize.so


In [ ]:
# ! wget http://images.cocodataset.org/zips/train2014.zip

In [60]:
## If 
# ! rm -fr cocoapi
# ! rm -fr pycocotools
# ! export CC="/usr/local/opt/llvm/bin/clang" &&\
#     git clone https://github.com/cocodataset/cocoapi.git &&\
#     cd cocoapi/PythonAPI &&\
#     ln -s cocoapi/PythonAPI/pycocotools/ pycocotools &&\
#     python3 setup.py build_ext --inplace

In [ ]:
! rm -fr coco
! rm -fr pycocotools
! export CC="/usr/local/opt/llvm/bin/clang" &&\
    git clone https://github.com/waleedka/coco.git &&\
    cd coco/PythonAPI &&\
    python3 setup.py build_ext --inplace

In [54]:
! rm -fr pycocotools
! ln -s coco/PythonAPI/pycocotools/ pycocotools

### Download the Model

Here:  https://drive.google.com/open?id=1VV6WgX_RNl6a9Yi9-Pe7ZyVKHRJZSKkm

And upload to the root directory of the repo on this machine (where this notebook lives).

## Setup and Run Inference

In [76]:
# Am I running 32 or 64 bit Python
import struct
print(struct.calcsize("P") * 8)

64


In [78]:
%load_ext autoreload
%autoreload
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import glob
from io import BytesIO
import requests
from PIL import Image
%matplotlib inline

# import coco
from pycocotools import coco
import utils
import model as modellib
import visualize

import torch
import pycocotools


# Root directory of the project
ROOT_DIR = os.getcwd()

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Path to trained weights file
# Download this file and place in the root of your
# project (See README file for details)
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "models", "mask_rcnn_coco.pth")

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    # GPU_COUNT = 0 for CPU
    GPU_COUNT = 0
    IMAGES_PER_GPU = 1
    COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.pth")


config = InferenceConfig()
config.display()

# Create model object.
model = modellib.MaskRCNN(model_dir=MODEL_DIR, config=config)
if config.GPU_COUNT:
    model = model.cuda()

# Load weights trained on MS-COCO
print(COCO_MODEL_PATH)
model.load_state_dict(torch.load(COCO_MODEL_PATH))

# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

# Load a random image from the images folder
# file_names = glob.glob(os.path.join('images', '*.jpg'))
# image = skimage.io.imread(os.path.join(random.choice(file_names)))

# Or load file from the internet
req = requests.get('http://mylongwalk.com/webdocs%20n%20lejog/Photographs/DN9%20Old%20car.jpg')
image = np.asarray(Image.open(BytesIO(req.content)))

# Run detection
results = model.detect([image])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ImportError: dlopen(/Users/micheleenharris/Documents/bin/github/pytorch-mask-rcnn-samples/nms/_ext/nms/_nms.so, 2): Symbol not found: ___cudaRegisterFatBinary
  Referenced from: /Users/micheleenharris/Documents/bin/github/pytorch-mask-rcnn-samples/nms/_ext/nms/_nms.so
  Expected in: flat namespace
 in /Users/micheleenharris/Documents/bin/github/pytorch-mask-rcnn-samples/nms/_ext/nms/_nms.so

In [75]:
# Visualize results
r = results[0]
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'],
                            class_names, r['scores'])
plt.show()

NameError: name 'results' is not defined

## Run an actual training experiment

In [ ]:
# ! python coco.py train --dataset=mask_rcnn_coco.pth --model=imagenet